In [1]:
import pandas as pd
import numpy as np
import pickle
import os
import seaborn as sns
import matplotlib.pyplot as plt
from gensim import matutils, models
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
from gensim.models.ldamodel import LdaModel
import scipy.sparse
import re
from gensim.utils import simple_preprocess
import spacy
import scispacy
import en_core_sci_lg
import gensim.corpora as corpora
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from pprint import pprint
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
from nltk.corpus import stopwords
import gensim.corpora as corpora

In [2]:
data=pd.read_csv("data.csv")
data.head()

,Unnamed: 0,Unnamed: 0.1,cord_uid,source_x,title,doi,abstract,publish_time,authors,journal,url,publish_year,lang,abstract_processed
0,20170,20170,u3s62cra,WHO,Necrotizing fasciitis and candidemia after toc...,NaN,INTRODUCTION: Tocilizumab (TCZ) is an interleu...,2021-01-01,"Setliff, E.; Kosmisky, D.; Ngeve, R.",Critical Care Medicine,NaN,2021,en,introduction tocilizumab (tcz) is an interleuk...
1,33573,33573,ign2rpta,Medline; PMC,Tisch Cancer Institute Scholars Program: Mento...,10.1007/s13187-020-01934-8,Cancer research has led to unprecedented advan...,2021-01-06,"Ben-David, Kaitlyn; Lin, Jenny J.; Ferrara, Ja...",J Cancer Educ,https://doi.org/10.1007/s13187-020-01934-8; ht...,2021,en,cancer research has led to unprecedented advan...
2,38733,38733,nvy3lqya,Medline; PMC,Prognostic Value of a Clinical Biochemistry-Ba...,10.3389/fmed.2020.597791,Background: This study aimed to explore the pr...,2021-01-18,"Yu, Jing; Nie, Lei; Wu, Dongde; Chen, Jian; Ya...",Front Med (Lausanne),https://doi.org/10.3389/fmed.2020.597791; http...,2021,en,background this study aimed to explore the pre...
3,85663,85663,w6f3xigb,Medline; PMC,Guillain-Barre syndrome in 220 patients with C...,10.1186/s41983-021-00310-7,This review summarises and discusses recent fi...,2021-05-04,"Finsterer, Josef; Scorza, Fulvio A.",Egypt J Neurol Psychiatr Neurosurg,https://www.ncbi.nlm.nih.gov/pubmed/33967575/;...,2021,en,this review summarises and discusses recent fi...
4,37078,37078,0vha3dkc,MedRxiv; WHO,The 2020 SARS-CoV-2 epidemic in England: key e...,10.1101/2021.01.11.21249564,We fitted a model of SARS-CoV-2 transmission i...,2021-01-13,"Knock, E. S.; Whittles, L. K.; Lees, J. A.; Pe...",NaN,http://medrxiv.org/cgi/content/short/2021.01.1...,2021,en,we fitted a model of sars-cov-2 transmission i...


In [5]:
#data = data.sample(100)
data=data

In [6]:
def sent_to_words(sentences):
    for sentence in sentences: yield(simple_preprocess(str(sentence), deacc=True))  

data = data.abstract_processed.values.tolist()
words = list(sent_to_words(data))


['the', 'administrative', 'accountability', 'system', 'in', 'china', 'has', 'evolved', 'during', 'public', 'health', 'crises', 'by', 'holding', 'public', 'officials', 'accountable', 'the', 'system', 'hopes', 'to', 'improve', 'their', 'performance', 'during', 'emergencies', 'however', 'covid', 'exposed', 'an']


# Bi-grams and Tri-grams

In [7]:
Bi = models.Phrases(words, min_count=6, threshold=100)
Tri = models.Phrases(Bi[words], threshold=100)
Bi_mod = models.phrases.Phraser(Bi)
Tri_mod = models.phrases.Phraser(Tri)

In [7]:
nlp = spacy.load("en_core_sci_lg", disable=['ner','tagger','parser','textcat'])
nlp.max_length = 3000000
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['sars_cov','patient','doi', 'preprint', 'copyright',  'biorxiv', 'from', 'subject', 're', 'ie', 'st','peer', 'reviewed', 'org', 'https', 'et', 'al', 'author', 'figure', 
    'rights', 'reserved', 'permission', 'used', 'using', 'th', 'rd', 'study', 'covid','covid-19', 'sars-cov-2', 'coronavirus', 'medrxiv', 'license', 'fig', 
    'al.', 'Elsevier', 'PMC', 'CZI'])

# resource https://www.kaggle.com/danielwolffram/topic-modeling-finding-related-articles



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15107\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
def remove_stopwords(texts):
    ans= [[word for word in simple_preprocess(str(papers)) if word not in stop_words] for papers in content]
    return ans    

def make_bigrams(texts):
    ans=[Bi_mod[papers] for papers in content]
    return ans

def make_trigrams(texts):
    ans= [Tri_mod[Bi_mod[papers]] for papers in content]
    return ans

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    ans = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc \
                          if token.pos_ in allowed_postags])
    return ans

In [10]:
nostops = remove_stopwords(data)

bigrams = make_bigrams(nostops)

lem = lemmatization(bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])


[['background', 'disease', 'pandemic', 'public', 'health', 'emergency', 'affect', 'many', 'world', 'nation', 'include', 'ethiopia', 'aside', 'implication', 'community', 'whole', 'also', 'associate', 'variety', 'mental', 'health', 'problem', 'healthcare', 'worker', 'hcw', 'aim', 'assess', 'prevalence', 'self', 'report', 'symptom', 'depression_anxiety', 'stress', 'pandemic', 'hcw', 'central', 'ethiopia', 'method', 'institutional', 'base', 'cross_sectional', 'conducted', 'use', 'online', 'survey', 'july', 'ethiopia', 'datum', 'collect', 'use', 'self', 'report', 'depression_anxiety', 'stress', 'scale', 'dass', 'datum', 'clean', 'code', 'analyse', 'use', 'spss', 'version', 'multivariable', 'logistic_regression', 'analysis', 'do', 'identify', 'associate', 'factor', 'mental', 'health', 'outcome', 'value', 'less', 'result', 'total', 'hcw', 'complete', 'self', 'report', 'questionnaire', 'percentage', 'hcw', 'moderate', 'extremely', 'severe', 'symptom', 'depression_anxiety', 'stress', 'respectiv

In [13]:
id2word = corpora.Dictionary(lem)
texts = lem
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 2), (23, 6), (24, 1), (25, 1), (26, 1), (27, 1), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 9), (35, 1), (36, 4), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 2), (47, 1), (48, 2), (49, 1), (50, 2), (51, 1), (52, 1), (53, 2), (54, 2), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 2), (63, 1), (64, 3), (65, 2), (66, 1), (67, 2), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 4), (74, 1), (75, 1), (76, 1), (77, 3), (78, 1), (79, 2), (80, 2), (81, 1), (82, 1), (83, 6), (84, 1), (85, 5), (86, 1), (87, 2), (88, 3), (89, 1), (90, 1), (91, 1), (92, 1), (93, 2), (94, 1), (95, 1), (96, 2)]]


In [14]:
pickle.dump(id2word, open("id2word", "wb"))
pickle.dump(corpus, open("corpus", "wb"))

# Number of Topic

In [15]:
def coherence_values(dic, corpus, content, limit, start=2, step=3):
    model = []
    values = []
    for topics in range(start, limit, step):
        model = models.LdaMulticore(corpus=corpus, num_topics=topics, id2word=dic)
        model.append(model)
        coherencemodel = CoherenceModel(model=model, texts=content, dictionary=dic, coherence='c_v')
        values.append(coherencemodel.get_coherence())
    return model, values

In [16]:
model, values = coherence_values(dictionary=dic, corpus=corpus, texts=data_lem, start=4, limit=30, step=2)

In [ ]:
fig,ax = plt.subplots(figsize=(8,5))
ax.plot(range(1,14),values,color='black')

In [ ]:
best = values.index(max(values))
best_model = model[best]


# Topic Model

In [19]:
lda_model = models.LdaMulticore(corpus=corpus,
                               id2word=id2word,
                               num_topics=26,
                               random_state=100,
                               chunksize=100,
                               passes=10)

In [21]:
topics=[]
for i in range(0,26):
    topiclist=[]
    for pharase, w in lda_model.show_topic(i, topn=20):
        topiclist.append((pharase, round(w,3)))
    topics.append(topiclist)

In [22]:
df = pd.DataFrame([', '.join([pharase for pharase, w in topic]) for topic in topics], columns = ['Terms per Topic'], index=['Topic'+str(t) for t in range(1, lda_model.num_topics+1)] )
pd.set_option('display.max_colwidth', -1)
df

c:\users\15107\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """


,Terms per Topic
Topic1,"quality, identify, drug, result, job_loss, study, use, intervention, sars_cov, ace, significant, difference, work, receptor, antagonist, histamine, good, prevention, ability, absenteeism"
Topic2,"student, model, use, result, cancer, first, clinical, method, pandemic, article, learning, technique, base, datum, patient, responder, programme, segment, stage, cell"
Topic3,"drive, model, clinic, simulation, vaccine, provide, design, world, large, helpful, louisville, vaccination, health, pandemic, result, third, use, resident, test, help"
Topic4,"bat, time, bat_adenovirus, divergence, first, patient, year, japan, estimate, datum, ago, guanylurea, sars_cov, cave, gene, china, approximately, eastern, large, test"
Topic5,"cancer, implementation, sars_cov, ppe, implementation_science, study, opticc, optimize, ebi, method, mgkg, develop, control, death, aim, lung, center, core, pe, risk"
Topic6,"patient, discharge, mirna, physical, msc, control, adsorption, infusion, time, uterine, group, severe, life, follow, protein, high, pulmonary, ard, score, determine"
Topic7,"system, food, pandemic, response, delivery, effect, asia, receptor, nanoparticle, vaccine, paper, antigen, environment, viral, adjuvant, manner, vector, virus, review, host"
Topic8,"health, patient, test, sars_cov, age, datum, disease, resident, infection, pandemic, mutation, plasma, hospital, mental, response, cluster, airway, outbreak, staff, nh"
Topic9,"diagnostic, tamcj, patient, deposit, resident, radiology, medicine, chest, ray, cr, complement_activation, case, mcj, family, gene, condition, protein, subfamily, interpretation, mcjs"
Topic10,"sars_cov, cancer, research, newborn, method, infection, sarcopenic_obesity, include, confirm, medical, mother, program, use, clinical, bear, evidence, type, care, provide, need"


In [ ]:
wc = WordCloud(background_color="white", max_font_size=150, random_state=42)
plt.rcParams['figure.figsize'] = [100, 100]
for i in range(26):
    wc.generate(text=df["Terms per Topic"][i])
    plt.subplot(26, 1, i+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(df.index[i], size=30)
plt.show()